# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "FTPMicrodadosNovoCaged"
extrator_bronze = AlinharETL(bucket,datamart)

2024-09-25 19:04:03,774 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [3]:
df = extrator_bronze.criar_view_temporaria('bronze/NovoCaged/MicrodadosNovoCagedIndicadorAprendiz','compliance_indicadoraprendiz')

2024-09-25 19:04:08,527 - INFO - Aguarde... Criando View (bronze/NovoCaged/MicrodadosNovoCagedIndicadorAprendiz)
2024-09-25 19:05:36,232 - INFO - View criada com sucesso


# Tratamentos na tabela 

In [4]:
sql_query = """
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_indicadoraprendiz
"""

In [5]:
df = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-25 19:05:36,244 - INFO - Aguarde... Executando Query (Delta)
2024-09-25 19:05:36,244 - INFO - 
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_indicadoraprendiz

2024-09-25 19:05:36,324 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [6]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/NovoCaged/MicrodadosNovoCagedIndicadorAprendiz'

In [7]:
extrator_bronze.salvar_delta(df, 'overwrite')

2024-09-25 19:05:36,341 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:05:45,252 - INFO - Dados persistidos com sucesso
2024-09-25 19:05:45,252 - INFO - s3a://silver/NovoCaged/MicrodadosNovoCagedIndicadorAprendiz
2024-09-25 19:05:45,253 - INFO - Aguarde... Realizando optimize
2024-09-25 19:05:49,887 - INFO - Optimize executado com sucesso.
2024-09-25 19:05:49,888 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:06:07,410 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [8]:
extrator_bronze.parar_sessao()

2024-09-25 19:06:07,883 - INFO - Sessão Spark finalizada.
